In [ ]:
''' 
PPCA v1.0.5

APPENDIX A1 : SLOPE CALCULATION AND CLUSTERING FOR CATCHMENT AREAS

Author: Perez, Joan

This script computes population estimates and slopes across four catchment distances using log-transformed population values. It clusters 
points by combining hierarchical clustering (for initial cluster centers) with k-means clustering, using the three slopes and the 
log-transformed population value at the second distance. Outputs include clustering results and visualizations of population curves 
and silhouette scores.

Full description, metadata, and output descriptions available here:
https://github.com/perezjoan/Population-Potential-on-Catchment-Area---PPCA-Worldwide/tree/main

Acknowledgement
This resource was produced within the emc2 project, which is funded by ANR (France), FFG (Austria), MUR (Italy) and Vinnova (Sweden) under
the Driving Urban Transition Partnership, which has been co-funded by the European Commission.

License: Attribution-ShareAlike 4.0 International - CC-BY-SA-4.0 license
''' 

In [ ]:
#---------------------------------------------------------------------------------#
# 0.1 : Box to fil with informations                                              #
# Name of the case study - Expected format : 'Nice'                               #
Name = ''                                                                         #
                                                                                  #
# Define the distances vector                                                     #
import numpy as np                                                                #
distances = np.array([160, 400, 800, 1200])   # <- need 4 distances here          #
#---------------------------------------------------------------------------------#
# 0.2 : Libraries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import sys
from sklearn.metrics import pairwise_distances
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import pdist, squareform
import gc
from sklearn.metrics import silhouette_score

# 0.3 Data preparation
gpkg = f'PPCA_4-1_{Name}_POP_CAT.gpkg'
nodes = gpd.read_file(gpkg, layer='points_catchment_stats')

In [ ]:
# 1. SLOPES EXTRACTION PER ROW

# Dynamically create column names based on the distances vector
columns_to_keep = [f'cc_Pop_estimation_sum_{dist}_nw' for dist in distances]

# Copy nodes DataFrame and create a new GeoDataFrame with only selected columns and geometry
nodes_selected = gpd.GeoDataFrame(nodes[columns_to_keep], geometry=nodes.geometry)

# Add 1 to each of the specified columns in nodes before log transformation (to avoid 0)
nodes_selected[columns_to_keep] = nodes_selected[columns_to_keep] + 1

# Create log-transformed columns in nodes_selected
for col in columns_to_keep:
    nodes_selected[f'log_{col}'] = np.log(nodes_selected[col])

# Log-transformed distances
x_points = np.log(distances)

# Function to calculate r2, beta, alpha (intercept), y estimates, and slopes for each row
def calculate_slopes(row):
    # y-values: log-transformed population estimates (use dynamically created column names)
    y_values = [row[f'log_cc_Pop_estimation_sum_{dist}_nw'] for dist in distances]
    
    # Reshape x_points
    X = x_points.reshape(-1, 1)
    
    # Calculate slopes between each consecutive pair of y estimates
    slope_1_2 = (y_values[1] - y_values[0]) / (x_points[1] - x_points[0])
    slope_2_3 = (y_values[2] - y_values[1]) / (x_points[2] - x_points[1])
    slope_3_4 = (y_values[3] - y_values[2]) / (x_points[3] - x_points[2])
    
    # Return results as a Series
    return pd.Series({
        'slope_1_2': slope_1_2,
        'slope_2_3': slope_2_3,
        'slope_3_4': slope_3_4
    })

# Split nodes_selected into batches of 1000 for progress tracking
batch_size = 1000
for start in range(0, len(nodes_selected), batch_size):
    end = min(start + batch_size, len(nodes_selected))
    nodes_selected.loc[start:end, ['slope_1_2', 'slope_2_3', 'slope_3_4']] = nodes_selected.iloc[start:end].apply(calculate_slopes, 
                                                                                                                 axis=1)
    
    # Progress display
    progress = (end / len(nodes_selected)) * 100
    sys.stdout.write(f"\rProcessed {progress:.2f}% of rows")
    sys.stdout.flush()

sys.stdout.write("\rProcessed 100.00% of rows\n")

In [ ]:
# 2. HIERARCHICAL CLUSTERING ON SAMPLE (TO DEFINE CLUSTER CENTERS START)

# Sampling 50,000 nodes for clustering
sample_size = 50000  # Adjust this as needed
sampled_nodes = nodes_selected.sample(n=sample_size, random_state=42)

# Define the clustering features (intercept, slopes, and log-transformed y-values)
features_sampled = sampled_nodes[['slope_1_2', 'slope_2_3', 'slope_3_4', 
                                  f'log_cc_Pop_estimation_sum_{distances[1]}_nw']]

# Scale the features
scaler = MinMaxScaler()
scaled_features_sampled = scaler.fit_transform(features_sampled)

# Perform hierarchical clustering directly with the scaled features
linkage_matrix_sampled = linkage(scaled_features_sampled, method='ward')

# Calculate the merge distances for clusters between 5 and 20
distances2 = linkage_matrix_sampled[:, 2]  # The third column contains the linkage distances
desired_clusters = np.arange(5, 21)  # Cluster range from 5 to 20
merge_distances = [distances2[len(distances2) - cluster_count] for cluster_count in desired_clusters]

# Calculate the absolute differences between successive merge distances
gap_differences = np.diff(merge_distances)

# Identify the indices of the top 3 largest gaps using absolute values
largest_gaps_indices = np.argsort(np.abs(gap_differences))[-3:][::-1]  # Top 3 largest gaps in descending order

# Get the merge distances corresponding to the largest gaps
top_merge_distances = [merge_distances[i] for i in largest_gaps_indices]

# Validate the cluster counts for each merge distance
cluster_counts = [
    len(np.unique(fcluster(linkage_matrix_sampled, t=merge_distance, criterion='distance')))
    for merge_distance in top_merge_distances
]

# Plot the dendrogram
plt.figure(figsize=(10, 7))
dendrogram(linkage_matrix_sampled, truncate_mode='level', p=15)

# Add horizontal lines for the top merge distances
for merge_distance in top_merge_distances:
    plt.axhline(y=merge_distance, color='r', linestyle='--', linewidth=1)

# Title and labels
plt.title("Dendrogram with Top 3 Largest Gaps")
plt.xlabel("")  # Remove x-axis label
plt.xticks([])  # Remove all x-axis ticks and labels
plt.ylabel("Distance")
plt.show()

# Get the number of clusters for each top merge distance
cluster_counts = [
    len(np.unique(fcluster(linkage_matrix_sampled, t=merge_distance, criterion='distance'))) + 1
    for merge_distance in top_merge_distances
]

# Print the number of clusters for each merge distance
print("Number of clusters corresponding to each threshold (top lines):", cluster_counts)

In [ ]:
# 3. KMEANS CLUSTERING WITH DEFINED STARTING CENTERS

# Function to get initial medoids based on hierarchical clustering
def get_sampled_initial_centers(features_sampled, dist_matrix, n_clusters):
    cluster_labels_sampled = fcluster(linkage_matrix_sampled, n_clusters, criterion='maxclust')
    initial_centers = []
    for cluster_id in range(1, n_clusters + 1):
        cluster_indices = np.where(cluster_labels_sampled == cluster_id)[0]
        cluster_center = features_sampled.iloc[cluster_indices].mean(axis=0).to_numpy()
        initial_centers.append(cluster_center)
    return np.array(initial_centers)

# Ensure full_features uses the entire nodes_selected dataset, not the sampled nodes
full_features = nodes_selected[['slope_1_2', 'slope_2_3', 'slope_3_4', 
                                f'log_cc_Pop_estimation_sum_{distances[1]}_nw']]
scaled_full_features = scaler.transform(full_features)  # Scale the full dataset

# Compute pairwise distance matrix for sampled features
sampled_dist_matrix = squareform(pdist(scaled_features_sampled, metric='euclidean'))

# Perform K-Means clustering with initialized centers
def perform_kmeans_with_sampled_initialization(full_features, sampled_features, dist_matrix, n_clusters):
    # Get initial cluster centers from sampled data
    initial_centers = get_sampled_initial_centers(pd.DataFrame(sampled_features), dist_matrix, n_clusters)
    
    # Run K-Means on the full dataset using the initial centers
    kmeans = KMeans(n_clusters=n_clusters, init=initial_centers, n_init=1, random_state=42)
    kmeans.fit(full_features)  # Fit K-Means on the full dataset
    return kmeans.labels_  # Return labels for the full dataset

# Perform clustering for different cluster sizes
for n_clusters in np.arange(5, 20):
    print(f"Clustering for {n_clusters} clusters started...")
    labels = perform_kmeans_with_sampled_initialization(
        scaled_full_features,  # Full dataset
        scaled_features_sampled,  # Sampled dataset
        sampled_dist_matrix,  # Distance matrix of the sampled dataset
        n_clusters
    )
    # Add clustering results to the full GeoDataFrame
    nodes_selected[f'hckmeans_{n_clusters}_clusters'] = labels
    print(f"Clustering for {n_clusters} clusters completed.")


In [ ]:
# Define the range of cluster counts
n_clusters_vector = np.arange(5, 20)  # Cluster solutions from 5 to 19

# Initialize a dictionary to store silhouette scores
silhouette_scores = {}

# Sample the dataset to reduce computation time
sampled_data = nodes_selected.sample(n=10000, random_state=42)  # Adjust sample size as needed

# Compute silhouette scores for each clustering solution
for n_clusters in n_clusters_vector:
    # Retrieve cluster labels for the sampled data
    labels = sampled_data[f'hckmeans_{n_clusters}_clusters']
    
    # Compute the silhouette score using relevant feature columns
    silhouette_scores[n_clusters] = silhouette_score(
        sampled_data[['slope_1_2', 'slope_2_3', 'slope_3_4']].values,  # Feature matrix
        labels  # Cluster labels
    )

# Plot silhouette scores
plt.figure(figsize=(10, 6))
plt.plot(
    list(silhouette_scores.keys()), 
    list(silhouette_scores.values()), 
    marker='o', linestyle='-', color='b'
)
plt.title("Silhouette Scores for Different Cluster Counts")
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Score")
plt.xticks(n_clusters_vector)
plt.grid()
plt.show()

In [ ]:
# Define the features to be used in the line plot
features = [f'log_cc_Pop_estimation_sum_{distances[0]}_nw', f'log_cc_Pop_estimation_sum_{distances[1]}_nw',
           f'log_cc_Pop_estimation_sum_{distances[2]}_nw', f'log_cc_Pop_estimation_sum_{distances[3]}_nw']
num_vars = len(features)

# Create a scaled copy of the dataset for the line plots
data_selected = nodes_selected.copy()

# Define the function to plot feature curves for multiple cluster solutions
def plot_feature_curves_multiple(data, title_prefix, n_clusters_vector):
    # Custom y-ticks and labels
    y_ticks = np.arange(0, 10)  # Tick positions (0 to 9)
    y_labels = ['0', '1 (1)', '2 (5)', '3 (20)', '4 (50)', '5 (150)', '6 (400)', '7 (1200)', '8 (3000)', '9 (8000)']

    for n_clusters in n_clusters_vector:
        cluster_column = f'hckmeans_{n_clusters}_clusters'

        # Compute the average of each feature per cluster in the specified column
        cluster_means = data.groupby(cluster_column)[features].mean()

        # Set up a single plot
        plt.figure(figsize=(10, 6))

        # Plot each feature's curve for each cluster
        for cluster_id, row in cluster_means.iterrows():
            # Plot each feature's value for a specific cluster, linking the points with a line
            plt.plot(features, row[features], label=f'Cluster {cluster_id}', marker='o', 
                     color=cluster_colors[cluster_id % len(cluster_colors)], linestyle='-', markersize=8)

        # Customize plot
        plt.title(f"{title_prefix} ({n_clusters} Clusters)", fontsize=16)
        plt.xlabel('Distance (meters)', fontsize=14)
        plt.ylabel('Population Values log (raw)', fontsize=14)

        # Rename x-axis labels to descriptive names
        x_labels = [f'{distances[0]} meters', f'{distances[1]} meters', f'{distances[2]} meters', f'{distances[3]} meters']
        plt.xticks(ticks=np.arange(len(features)), labels=x_labels, rotation=45)

        # Set y-axis ticks and labels
        plt.yticks(ticks=y_ticks, labels=y_labels)

        plt.legend(loc='upper left', bbox_to_anchor=(1, 1), title='Cluster')
        plt.tight_layout()  # Adjust layout to prevent overlap
        plt.show()

# Generate plots for multiple cluster solutions
plot_feature_curves_multiple(data_selected, title_prefix="Feature Curves", n_clusters_vector=n_clusters_vector)


In [ ]:
# 5. CURVES ANALYSIS : SLOPES

# Define the features to be used in the line plot
features = [f'log_cc_Pop_estimation_sum_{distances[1]}_nw', 'slope_1_2', 'slope_2_3', 'slope_3_4']
num_vars = len(features)

# Create a scaled copy of the dataset for the line plots
data_selected = nodes_selected.copy()

# Define a list of colors for up to 20 clusters
cluster_colors = [
    'grey', 'lightgreen', 'darkgreen', 'lightblue', 'yellow', 
    'orange', 'mediumblue', 'darkblue', 'red', 'brown',
    'purple', 'pink', 'cyan', 'olive', 'teal', 
    'gold', 'navy', 'lime', 'maroon', 'turquoise'
]

# Define a function to plot the feature curves for multiple cluster solutions
def plot_feature_curves_multiple(data, title_prefix, n_clusters_vector):
    for n_clusters in n_clusters_vector:
        cluster_column = f'hckmeans_{n_clusters}_clusters'

        # Compute the average of each feature per cluster in the specified column
        cluster_means = data.groupby(cluster_column)[features].mean()

        # Set up a single plot
        plt.figure(figsize=(10, 6))

        # Plot each feature's curve for each cluster
        for cluster_id, row in cluster_means.iterrows():
            # Plot each feature's value for a specific cluster, linking the points with a line
            plt.plot(features, row[features], label=f'Cluster {cluster_id}', marker='o', 
                     color=cluster_colors[cluster_id % len(cluster_colors)], linestyle='-', markersize=8)

        # Customize plot
        plt.title(f"{title_prefix} ({n_clusters} Clusters)", fontsize=16)
        plt.xlabel('', fontsize=14)
        plt.ylabel('Values', fontsize=14)

        # Rename x-axis labels to more descriptive names
        x_labels = ['pop_400(log)', 'slope 1-2', 'slope 2-3', 'slope 3-4']
        plt.xticks(ticks=np.arange(len(features)), labels=x_labels, rotation=45)

        plt.legend(loc='upper left', bbox_to_anchor=(1, 1), title='Cluster')
        plt.tight_layout()  # Adjust layout to prevent overlap
        plt.show()

# Generate plots for multiple cluster solutions
plot_feature_curves_multiple(data_selected, title_prefix="Feature Curves", n_clusters_vector=n_clusters_vector)

In [ ]:
###########################################################################################################################################

## APPENDICES

# A1. Save Outputs
gpkg = f'PPCA_A1_{Name}.gpkg'
nodes_selected.to_file(gpkg, layer='Points_with_reg_and_hckmeans', driver="GPKG")